In [1]:
# Problem 1

import requests
import pandas as pd
import json
from pandas.io.json import json_normalize

# I check if the connection is successful printing the status. Ok if the response is 200
ra = requests.get("https://testapp.deribit.com/api/v2/public/auth?grant_type=client_credentials&client_id=dJAnpAEn&client_secret=tuOL8RSCrM3HZMliPxbg9ja4ghViCqx8-dA5o3U63aY")
print (ra)
# I request the trades for the account
re = requests.get("https://testapp.deribit.com/api/v2/private/get_user_trades_by_currency?currency=BTC", auth = ('dJAnpAEn','tuOL8RSCrM3HZMliPxbg9ja4ghViCqx8-dA5o3U63aY'))
df = json_normalize (re.json())
# I convert the json in DataFrame
df_trades = json_normalize (df['result.trades'].iloc[0])
df_trades

<Response [200]>


,trade_seq,trade_id,timestamp,tick_direction,state,self_trade,reduce_only,price,post_only,order_type,...,matching_id,liquidity,iv,instrument_name,index_price,fee_currency,fee,direction,amount,original_order_type
0,29,26079521,1573485086101,3,filled,False,False,0.055,False,limit,...,None,M,103.02,BTC-15NOV19-8500-C,8717.84,BTC,0.000400,buy,1.0,NaN
1,24,25831959,1573393037334,0,filled,False,False,0.082,False,limit,...,None,M,136.31,BTC-15NOV19-8500-C,8830.97,BTC,0.000400,buy,1.0,market
2,14525923,25830820,1573390905649,0,filled,False,False,8842.000,False,limit,...,None,T,NaN,BTC-PERPETUAL,8841.33,BTC,0.000131,buy,1550.0,NaN
3,14525593,25830448,1573390059018,0,filled,False,False,8856.000,False,limit,...,None,M,NaN,BTC-PERPETUAL,8855.97,BTC,-0.001335,sell,47280.0,NaN
4,2466887,25830324,1573389598829,0,filled,False,False,8935.500,False,limit,...,None,T,NaN,BTC-27DEC19,8853.82,BTC,0.000071,sell,1260.0,NaN
5,14486212,25788395,1573333967084,0,filled,False,True,8788.500,False,market,...,None,T,NaN,BTC-PERPETUAL,8785.05,BTC,0.000594,buy,6960.0,NaN
6,14486211,25788394,1573333967084,1,filled,False,True,8788.000,False,market,...,None,T,NaN,BTC-PERPETUAL,8785.05,BTC,0.002548,buy,29850.0,NaN
7,14486210,25788393,1573333967084,1,filled,False,True,8788.000,False,market,...,None,T,NaN,BTC-PERPETUAL,8785.05,BTC,0.000009,buy,100.0,NaN
8,14486209,25788392,1573333967084,0,filled,False,True,8788.000,False,market,...,None,T,NaN,BTC-PERPETUAL,8785.05,BTC,0.000009,buy,100.0,NaN
9,14486208,25788391,1573333967084,1,filled,False,True,8787.500,False,market,...,None,T,NaN,BTC-PERPETUAL,8785.05,BTC,0.002466,buy,28890.0,NaN


In [2]:
# Problem 2

from scipy import stats
import numpy as np

# Function computing d1
def d1 (S, K, T, r, d, sigma):
    return (np.log(S/K) + ((r-d) + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

# Function computing d2
def d2 (S, K, T, r, d, sigma):
    return d1(S, K, T, r, d, sigma) - sigma * np.sqrt(T)

# Function computing the call
def call (S, K, T, r, d, sigma) :
    return S * np.exp(-d*T)* stats.norm(0,1).cdf(d1(S, K, T, r, d, sigma)) - K * np.exp(-r * T) * stats.norm(0,1).cdf(d2(S, K, T, r, d, sigma))

In [3]:
# Let's try to calculate a call
call(100, 80, 1, 0.05, 0.0, 0.3)

26.462085709671783

In [4]:
# Problem 3

import pandas as pd
import numpy as np
import time
import datetime

# We read the csv file
df = './SAMPLE_transactions.csv'
data = pd.read_csv(df)

# Question 1

print (data.loc[:,["account","instrument","price"]].groupby(["account","instrument"]).mean())

                                       price
account instrument                          
Test_1  TEST.BTC                    0.000000
        TEST.BTC-25OCT19-7250-C     0.150833
        TEST.BTC-25OCT19-7500-C     0.129000
        TEST.BTC-25OCT19-8000-C     0.088833
        TEST.BTC-27DEC19         8384.000000
        TEST.BTC-27DEC19-8000-C     0.190500
        TEST.BTC-27DEC19-9000-C     0.141500
        TEST.BTC-27DEC19-9000-P     0.199000
        TEST.BTC-PERPETUAL       8315.235294
        TEST.ETH                    0.000000
        TEST.ETH-27MAR20-240-C      0.097000
Test_2  TEST.BTC-25OCT19-7250-C     0.152833
        TEST.BTC-25OCT19-7500-C     0.130167
        TEST.BTC-25OCT19-8500-C     0.059667
        TEST.BTC-PERPETUAL       8331.500000


In [5]:
# Question 2

# A first possibility
print (data.loc[:,["instrument","price"]].groupby("instrument").get_group("TEST.BTC-PERPETUAL").agg([np.mean,np.median,np.std]))

# A second possibility
print (data[data.loc[:, 'instrument'] == 'TEST.BTC-PERPETUAL'].price.agg([np.mean,np.median,np.std]))

              price
mean    8320.865385
median  8320.750000
std       14.119177
mean      8320.865385
median    8320.750000
std         14.119177
Name: price, dtype: float64


In [16]:
# Question 3

print (data.loc[:,["fees","currency"]].groupby("currency").sum())


              fees
currency          
BTC       0.023632
ETH       0.001600


In [17]:
# Question 4

# We create a new column 'date'
data['date'] = data.timestamp.apply(lambda x: str(pd.Timestamp(x/10e8, unit='s')).split()[0])

# We define 2 dates for comparison
target_date = "10/11/2019"
nextday = "10/12/2019"

target_stamp = time.mktime(datetime.datetime.strptime(target_date, "%m/%d/%Y").timetuple()) * 1e9
nextday_stamp = time.mktime(datetime.datetime.strptime(nextday, "%m/%d/%Y").timetuple()) * 1e9

# We check if the date is the one searched
data['is_target'] = data.timestamp.apply(lambda x: x >= target_stamp and x < nextday_stamp)

# We define a temporary df
df_tmp = data[data.loc[:,'is_target']]

# We create a final df with just Test_2
df_final = df_tmp[df_tmp.loc[:,'account'] == 'Test_2'].copy()

df_final

,id,instrument,price,quantity,fees,currency,contract_size,account,timestamp,type,payout,date,is_target
24,23676007,TEST.BTC-PERPETUAL,8334.0000,976.0,8.783300e-04,BTC,10,Test_2,1.570806e+18,TRADE,INVERSE,2019-10-11,True
25,23676025,TEST.BTC-25OCT19-8500-C,0.0600,-3.0,1.200000e-03,BTC,1,Test_2,1.570806e+18,TRADE,NORMAL,2019-10-11,True
26,23676026,TEST.BTC-25OCT19-7500-C,0.1305,-1.0,4.000000e-04,BTC,1,Test_2,1.570806e+18,TRADE,NORMAL,2019-10-11,True
27,23676027,TEST.BTC-PERPETUAL,8337.0000,1.0,9.000000e-07,BTC,10,Test_2,1.570806e+18,TRADE,INVERSE,2019-10-11,True
28,23676028,TEST.BTC-PERPETUAL,8337.0000,583.0,5.244700e-04,BTC,10,Test_2,1.570806e+18,TRADE,INVERSE,2019-10-11,True
29,23676029,TEST.BTC-25OCT19-7500-C,0.1305,-1.0,4.000000e-04,BTC,1,Test_2,1.570810e+18,TRADE,NORMAL,2019-10-11,True
30,23676030,TEST.BTC-PERPETUAL,8337.0000,584.0,5.253700e-04,BTC,10,Test_2,1.570810e+18,TRADE,INVERSE,2019-10-11,True
31,23676050,TEST.BTC-25OCT19-7250-C,0.1530,-1.0,4.000000e-04,BTC,1,Test_2,1.570806e+18,TRADE,NORMAL,2019-10-11,True
32,23676051,TEST.BTC-PERPETUAL,8334.5000,609.0,5.480200e-04,BTC,10,Test_2,1.570806e+18,TRADE,INVERSE,2019-10-11,True
33,23676052,TEST.BTC-25OCT19-7250-C,0.1530,-1.0,4.000000e-04,BTC,1,Test_2,1.570806e+18,TRADE,NORMAL,2019-10-11,True
